In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import json

import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
!pip install pyTigerGraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
import time
import random as rand
from pathlib import Path, PurePosixPath
import pyTigerGraph as tg
import os
from shutil import copyfile

In [ ]:
from platform import python_version
print(python_version())

3.10.12


In [ ]:
hostName = "localhost"                            # TG server hostname
userName = "tigergraph"                           # TG user name
passWord = "tigergraph"                           # TG password

In [ ]:
topK = 10                                         # Number of highest scoring drugs
numDevices = 1                                    # Number of FPGA devices to distribute the queries to

In [ ]:
localRepoLocation = Path("/opt/xilinx/apps")
exampleLocation = Path("graphanalytics/integration/Tigergraph-3.x/1.2/examples/drug_similarity/") # when running from github repo
queryFileLocation = localRepoLocation / exampleLocation / "query"

In [ ]:
lexique = pd.read_csv('LEX.csv')

In [ ]:
import pandas as pd

# Assuming that 'lexique' is already your dataframe
lexique.rename(columns={
    'Terme': 'Biomedical Term',
    'Taux de complexité': 'Complexity Rate',
    'Indice de comparaison': 'Comparison Index',
    'Explication au grand public': 'Public Explanation',
    'Taux de simplification': 'Simplification Rate',
    'Sources': 'Sources'
}, inplace=True)

# Check the changes by printing the new columns
print(lexique.columns)

Index(['Biomedical Term', 'Complexity Rate', 'Comparison Index',
       'Public Explanation', 'Simplification Rate', 'Sources'],
      dtype='object')


In [ ]:
mrconso = pd.read_csv('MRCONSO.csv')

In [ ]:
mrconso.columns = [
    'CUI (Concept Unique Identifier)', 'LAT (Language)', 'TS (Term Status)',
    'LUI (Lexical Unique Identifier)', 'STT (String Type)', 'SUI (String Unique Identifier)',
    'ISPREF (Is Preferred)', 'AUI (Atom Unique Identifier)', 'MetaUI (Metadata Unique Identifier)',
    'DescriptorUI (Descriptor Unique Identifier)', 'SAB (Source Abbreviation)', 'TTY (Term Type)',
    'Associated DescriptorUI (Descriptor Unique Identifier)', 'STR (String)', 'Frequency',
    'SUPPRESS (Suppressible Flag)'
]


In [ ]:
mrconso.head()

,CUI (Concept Unique Identifier),LAT (Language),TS (Term Status),LUI (Lexical Unique Identifier),STT (String Type),SUI (String Unique Identifier),ISPREF (Is Preferred),AUI (Atom Unique Identifier),MetaUI (Metadata Unique Identifier),DescriptorUI (Descriptor Unique Identifier),SAB (Source Abbreviation),TTY (Term Type),Associated DescriptorUI (Descriptor Unique Identifier),STR (String),Frequency,SUPPRESS (Suppressible Flag),STR (String) EN
0,C0000005,FRE,P,L6220710,PF,S7133957,Y,A13433185,M0019694,D012711,MSHFRE,PEP,D012711,Macroagrégats d'albumine marquée à l'iode 131,3,N,Macroaggregates of albumin labeled with iodine...
1,C0000005,FRE,S,L6215648,PF,S7133916,Y,A27488794,M0019694,D012711,MSHFRE,ET,D012711,MAA-I 131,3,N,MAA-I 131
2,C0000005,FRE,S,L6215656,PF,S7133956,Y,A27614225,M0019694,D012711,MSHFRE,ET,D012711,Macroagrégats d'albumine humaine marquée à l'i...,3,N,Macroaggregates of human albumin labeled with ...
3,C0000039,FRE,P,L3245036,PF,S3772540,Y,A7439429,M0023172,D015060,MSHFRE,MH,D015060,"1,2-Dipalmitoylphosphatidylcholine",3,N,"1,2-Dipalmitoylphosphatidylcholine"
4,C0000052,FRE,P,L3245041,PF,S3772546,Y,A7439435,M0023173,D015061,MSHFRE,MH,D015061,"1,4-alpha-Glucan branching enzyme",3,N,"1,4-alpha-Glucan branching enzyme"


In [ ]:
# Installation of spaCy
!pip install -U spacy

# Downloading the model for French
!python -m spacy download fr_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.8/571.8 MB 1.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Installation of spaCy
!pip install -U spacy

# Downloading the model for English
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Step 0: Importing the library with the algorithm
#!pip install gensim if not already installed
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

from gensim import models
from gensim.models import Phrases
from gensim import corpora

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# IGNORE THIS CELL



import pandas as pd
import numpy as np
import re
import spacy
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Loading the data
lex_df = pd.read_csv('LEX.csv')
mrconso_df = pd.read_csv('MRCONSO.csv')

# Data cleaning and normalization
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Removing stopwords
def remove_stopwords(text, language='french'):
    stop_words = set(stopwords.words(language))
    words = text.split()
    return ' '.join([w for w in words if w not in stop_words])

# Initializing spaCy for the French language
nlp = spacy.load('fr_core_news_lg')

# Lemmatization of the data
def lemmatize_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

# Complete text preprocessing function
def preprocess_text(text):
    text = clean_text(text)
    text = remove_stopwords(text)
    text = lemmatize_text(text)
    return text

# Applying preprocessing directly to the relevant columns
lex_df['Cleaned'] = lex_df['Source'].apply(preprocess_text)
mrconso_df['Cleaned'] = mrconso_df['Cible'].apply(preprocess_text)

# Preparing data for similarity computation
# Temporary concatenation of the preprocessed columns 'Terme' and 'Explication au grand public'
combined_lex = lex_df['Cleaned'].tolist()
cleaned_mrconso = mrconso_df['Cleaned'].tolist()

In [ ]:
import pandas as pd
import numpy as np
import re
import spacy
from nltk.corpus import stopwords
from nltk.stem.snowball import EnglishStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Loading data
lex_df = pd.read_csv('LEX.csv')
mrconso_df = pd.read_csv('MRCONSO.csv')

# Text cleaning and normalization
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Stopwords removal
def remove_stopwords(text, language='english'):
    stop_words = set(stopwords.words(language))
    words = text.split()
    return ' '.join([w for w in words if w not in stop_words])

# Initialize spaCy for English biomedical language
nlp = spacy.load('en_core_web_lg')

# Lemmatizing the data
def lemmatize_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

# Complete text preprocessing function
def preprocess_text(text):
    text = clean_text(text)
    text = remove_stopwords(text)
    text = lemmatize_text(text)
    return text

# Applying preprocessing directly to the relevant columns
lex_df['Cleaned'] = lex_df['Source'].apply(preprocess_text)
mrconso_df['Cleaned'] = mrconso_df['Target'].apply(preprocess_text)

# Preparing data for similarity computation
# Temporary concatenation of the preprocessed columns 'Terme' and 'Explication au grand public'
combined_lex = lex_df['Cleaned'].tolist()
cleaned_mrconso = mrconso_df['Cleaned'].tolist()

In [ ]:
cleaned_mrconso

['macroaggregate albumin label iodine 131',
 'maa I 131',
 'macroaggregate human albumin label iodine 131',
 '1 2 dipalmitoylphosphatidylcholine',
 '1 4 alpha glucan branching enzyme',
 'branch enzyme',
 'branch enzyme',
 'amylo 1 4 1 6 transglycosylase',
 'alpha glucan branching glycosyltransferase',
 '2 acyl 1 alkyl phosphatidate',
 '1 alkyl 2 acyl phosphatidate',
 '2 acyl 1 alkylphosphatidates',
 '1 alkyl 2 acylphosphatidate',
 '1 carboxy glutamic acid',
 'gamma carboxyglutamic acid',
 '1 carboxyglutamic acid',
 'gamma carboxy glutamic acid',
 '3 aminopropane 1 1 3 tricarboxylic acid',
 '3 amino propane 1 1 3 tricarboxylic acid',
 'xanthine 3 isobutyl 1 methyl',
 '3 isobutyl 1 methylxanthine',
 '1 methyl 4 phenyl 1 2 3 6 tetrahydropyridine',
 'mptp',
 '1 methyl 4 phenyl pyridinium',
 '1 methyl 4 phenylpyridinium',
 '1 naphthylamine',
 '1 naphthyl isothiocyanate',
 '1 naphthylisothiocyanate',
 '1 sarcosyl 8 isoleucyl angiotensin ii',
 '1 sar 8 ile angiotensin ii',
 '1 sarcosyl 8 isol

In [ ]:
mrconso_df.head()

,CUI (Concept Unique Identifier),LAT (Language),TS (Term Status),LUI (Lexical Unique Identifier),STT (String Type),SUI (String Unique Identifier),ISPREF (Is Preferred),AUI (Atom Unique Identifier),MetaUI (Metadata Unique Identifier),DescriptorUI (Descriptor Unique Identifier),SAB (Source Abbreviation),TTY (Term Type),Associated DescriptorUI (Descriptor Unique Identifier),STR (String),Frequency,SUPPRESS (Suppressible Flag),STR (String) EN,Cleaned
0,C0000005,FRE,P,L6220710,PF,S7133957,Y,A13433185,M0019694,D012711,MSHFRE,PEP,D012711,Macroagrégats d'albumine marquée à l'iode 131,3,N,Macroaggregates of albumin labeled with iodine...,macroaggregate albumin label iodine 131
1,C0000005,FRE,S,L6215648,PF,S7133916,Y,A27488794,M0019694,D012711,MSHFRE,ET,D012711,MAA-I 131,3,N,MAA-I 131,maa I 131
2,C0000005,FRE,S,L6215656,PF,S7133956,Y,A27614225,M0019694,D012711,MSHFRE,ET,D012711,Macroagrégats d'albumine humaine marquée à l'i...,3,N,Macroaggregates of human albumin labeled with ...,macroaggregate human albumin label iodine 131
3,C0000039,FRE,P,L3245036,PF,S3772540,Y,A7439429,M0023172,D015060,MSHFRE,MH,D015060,"1,2-Dipalmitoylphosphatidylcholine",3,N,"1,2-Dipalmitoylphosphatidylcholine",1 2 dipalmitoylphosphatidylcholine
4,C0000052,FRE,P,L3245041,PF,S3772546,Y,A7439435,M0023173,D015061,MSHFRE,MH,D015061,"1,4-alpha-Glucan branching enzyme",3,N,"1,4-alpha-Glucan branching enzyme",1 4 alpha glucan branching enzyme


In [ ]:
# IGNORE THIS CELL



import pandas as pd
import numpy as np
import re
import spacy
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Loading the data
lex_df = pd.read_csv('LEX.csv')


# Data cleaning and normalization
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Removing stopwords
def remove_stopwords(text, language='french'):
    stop_words = set(stopwords.words(language))
    words = text.split()
    return ' '.join([w for w in words if w not in stop_words])

# Initializing spaCy for the French language
nlp = spacy.load('fr_core_news_lg')

# Lemmatization of the data
def lemmatize_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

# Complete text preprocessing function
def preprocess_text(text):
    text = clean_text(text)
    text = lemmatize_text(text)
    text = remove_stopwords(text)
    return text

# Applying preprocessing directly to the relevant columns
# # # # # # # # lex_df['Cleaned'] = lex_df['Biomedical Term'].apply(preprocess_text)
lex_df['Cleaned'] = lex_df['Public Explanation'].apply(preprocess_text)


# Preparing data for similarity computation
# Temporary concatenation of the preprocessed columns 'Terme' and 'Explication au grand public'
combined_lex = lex_df['Cleaned'].tolist()

In [ ]:
#x1 = pd.read_csv('lex_cleaned.csv')
mrconso_df = pd.read_csv('mrconso_cleaned.csv')

#combined_lex = x1['Cleaned'].tolist()
cleaned_mrconso = mrconso_df['Cleaned'].tolist()

In [ ]:
mrconso_df.head()

,CUI (Concept Unique Identifier),LAT (Language),TS (Term Status),LUI (Lexical Unique Identifier),STT (String Type),SUI (String Unique Identifier),ISPREF (Is Preferred),AUI (Atom Unique Identifier),MetaUI (Metadata Unique Identifier),DescriptorUI (Descriptor Unique Identifier),SAB (Source Abbreviation),TTY (Term Type),Associated DescriptorUI (Descriptor Unique Identifier),STR (String),Frequency,SUPPRESS (Suppressible Flag),Cleaned
0,C0000005,FRE,P,L6220710,PF,S7133957,Y,A13433185,M0019694,D012711,MSHFRE,PEP,D012711,Macroagrégats d'albumine marquée à l'iode 131,3,N,macroagrégat albumine marquer iode 131
1,C0000005,FRE,S,L6215648,PF,S7133916,Y,A27488794,M0019694,D012711,MSHFRE,ET,D012711,MAA-I 131,3,N,maa i 131
2,C0000005,FRE,S,L6215656,PF,S7133956,Y,A27614225,M0019694,D012711,MSHFRE,ET,D012711,Macroagrégats d'albumine humaine marquée à l'i...,3,N,macroagrégat albumine humain marquer iode 131
3,C0000039,FRE,P,L3245036,PF,S3772540,Y,A7439429,M0023172,D015060,MSHFRE,MH,D015060,"1,2-Dipalmitoylphosphatidylcholine",3,N,1 2 dipalmitoylphosphatidylcholine
4,C0000052,FRE,P,L3245041,PF,S3772546,Y,A7439435,M0023173,D015061,MSHFRE,MH,D015061,"1,4-alpha-Glucan branching enzyme",3,N,1 4 alpha glucan branching enzyme


In [ ]:
#x1 = pd.read_csv('lex_cleaned.csv')
#x2 = pd.read_csv('mrconso_cleaned.csv')

#combined_lex = x1['Cleaned'].tolist()
#cleaned_mrconso = x2['Cleaned'].tolist()

In [ ]:
# MAIN WAY



from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# It is assumed that lex_df and mrconso_df have already been defined and preprocessed in Script 1

# Loading the Tokenizer and the Model
tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-KRISSBERT-PubMed-UMLS-EL')
model = AutoModel.from_pretrained('microsoft/BiomedNLP-KRISSBERT-PubMed-UMLS-EL')

# Modification: Function to encode texts in batches
def encode_texts_in_batches(texts, batch_size=10):  # You can adjust the batch size according to your available memory
    model.eval()  # Set the model to evaluation mode
    embeddings_list = []  # List to store batch embeddings
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        with torch.no_grad():  # Do not calculate gradients to save memory and speed up computation
            encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
            output = model(**encoded_input)
        batch_embeddings = output.last_hidden_state.mean(dim=1)
        embeddings_list.append(batch_embeddings)
    embeddings = torch.cat(embeddings_list, dim=0)
    return embeddings

lex_embeddings = encode_texts_in_batches(combined_lex)
mrconso_embeddings = encode_texts_in_batches(cleaned_mrconso)

# Row-by-row cosine similarity computation
cosine_similarities = []
for i in range(len(lex_embeddings)):
    cosine_sim = cosine_similarity(lex_embeddings[i].unsqueeze(0).numpy(), mrconso_embeddings[i].unsqueeze(0).numpy())
    cosine_similarities.append(cosine_sim[0][0])

# Visualization of the results
for idx, cos_sim in enumerate(cosine_similarities):
    lex_term = lex_df.iloc[idx]['Source']
    matched_term = mrconso_df.iloc[idx]['Target']
    print(f"{lex_term} si allinea a {matched_term} con una similarità cosinus di {cos_sim:.2f}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In the absence of improvement or if symptoms persist, seek medical advice. si allinea a In the absence of improvement or if symptoms persist beyond 7 days of treatment, seek medical advice. con una similarità cosinus di 0.99
In the absence of improvement or if symptoms persist, seek medical advice. si allinea a In the absence of improvement or if symptoms persist, seek medical advice. con una similarità cosinus di 1.00
Additionally, the patient will be warned of the necessity of a rapid consultation in the event of any abnormal vaginal bleeding. si allinea a In case of abnormal vaginal bleeding, it is important to consult your doctor as soon as possible. con una similarità cosinus di 0.98
The administration of paraffin oil to young children, debilitated persons, bedridden individuals, or those with swallowing difficulties should be cautious due to the risk of bronchial aspiration and lipoid pneumonia. si allinea a The administration of paraffin oil in young children, bedridden patients

In [ ]:
# INGORE THIS CELL



from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# It is assumed that lex_df and mrconso_df have already been defined and preprocessed in Script 1

# Loading the Tokenizer and the Model
tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-KRISSBERT-PubMed-UMLS-EL')
model = AutoModel.from_pretrained('microsoft/BiomedNLP-KRISSBERT-PubMed-UMLS-EL')

# Modification: Function to encode texts in batches
def encode_texts_in_batches(texts, batch_size=10):  # You can adjust the batch size according to your available memory
    model.eval()  # Set the model to evaluation mode
    embeddings_list = []  # List to store batch embeddings
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        with torch.no_grad():  # Do not calculate gradients to save memory and speed up computation
            encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
            output = model(**encoded_input)
        batch_embeddings = output.last_hidden_state.mean(dim=1)
        embeddings_list.append(batch_embeddings)
    embeddings = torch.cat(embeddings_list, dim=0)
    return embeddings

# Applying batch encoding
lex_embeddings = encode_texts_in_batches(combined_lex)
mrconso_embeddings = encode_texts_in_batches(cleaned_mrconso)

# Cosine similarity computation
cosine_sim = cosine_similarity(lex_embeddings.numpy(), mrconso_embeddings.numpy())

# Identification of the best matches
top_matches = np.argmax(cosine_sim, axis=1)
for idx, match_idx in enumerate(top_matches):
    lex_term = lex_df.iloc[idx]['Biomedical Term']
    lex_explication = lex_df.iloc[idx]['Public Explanation']
    matched_term = mrconso_df.iloc[match_idx]['STR (String)']
    print(f"{lex_term} ({lex_explication}) s'aligne à {matched_term} avec une similarité cosinus de {cosine_sim[idx, match_idx]:.2f}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Vascularite (Inflammation des petits vaisseaux sanguins) s'aligne à Vasculite des petits vaisseaux cutanés avec une similarité cosinus de 0.99
Vascularite (Inflammation des vaisseaux) s'aligne à Inflammation du poumon avec une similarité cosinus de 0.99
Vascularite (Inflammation des vaisseaux) s'aligne à Inflammation du poumon avec une similarité cosinus de 0.99
Vascularite (Inflammation des vaisseaux pouvant se traduire par une éruption cutanée) s'aligne à Vascularite SAI avec une similarité cosinus de 0.98
Vascularite (Inflammation des vaisseaux sanguins) s'aligne à Vasculite des petits vaisseaux cutanés avec une similarité cosinus de 0.99
Vascularite (Inflammation des vaisseaux sanguins) s'aligne à Vasculite des petits vaisseaux cutanés avec une similarité cosinus de 0.99
Vascularite (Inflammation des vaisseaux sanguins) s'aligne à Vasculite des petits vaisseaux cutanés avec une similarité cosinus de 0.99


In [ ]:
# IGNORE THIS CELL



from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# It is assumed that lex_df and mrconso_df have already been defined and preprocessed in Script 1

# Loading the Tokenizer and the Model
tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-KRISSBERT-PubMed-UMLS-EL')
model = AutoModel.from_pretrained('microsoft/BiomedNLP-KRISSBERT-PubMed-UMLS-EL')

# Modification: Function to encode texts in batches
def encode_textss(texts):
    model.eval()  # # Set the model to evaluation mode
    with torch.no_grad():  # Do not calculate gradients to save memory and speed up computation
        encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
        output = model(**encoded_input)
        embeddings = output.last_hidden_state.mean(dim=1)
    return embeddings

# Applying batch encoding
lex_embeddingss = encode_textss(combined_lex)
mrconso_embeddingss = encode_textss(cleaned_mrconso)

# Row-by-row cosine similarity computation
cosine_similaritiess = []
for i in range(len(lex_embeddingss)):
    cosine_sim = cosine_similarity(lex_embeddingss[i].unsqueeze(0).numpy(), mrconso_embeddingss[i].unsqueeze(0).numpy())
    cosine_similaritiess.append(cosine_sim[0][0])

# Visualization of the results
for idx, cos_sim in enumerate(cosine_similaritiess):
    lex_term = lex_df.iloc[idx]['Source']
    matched_term = mrconso_df.iloc[idx]['Target']
    print(f"{lex_term} si allinea a {matched_term} con una similarità cosinus di {cos_sim:.2f}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In the absence of improvement or if symptoms persist, seek medical advice. si allinea a In the absence of improvement or if symptoms persist beyond 7 days of treatment, seek medical advice. con una similarità cosinus di 0.99
In the absence of improvement or if symptoms persist, seek medical advice. si allinea a In the absence of improvement or if symptoms persist, seek medical advice. con una similarità cosinus di 1.00
Additionally, the patient will be warned of the necessity of a rapid consultation in the event of any abnormal vaginal bleeding. si allinea a In case of abnormal vaginal bleeding, it is important to consult your doctor as soon as possible. con una similarità cosinus di 0.98
The administration of paraffin oil to young children, debilitated persons, bedridden individuals, or those with swallowing difficulties should be cautious due to the risk of bronchial aspiration and lipoid pneumonia. si allinea a The administration of paraffin oil in young children, bedridden patients

In [ ]:
import pandas as pd

# Assuming that top_matches, cosine_sim, lex_df, and mrconso_df are already defined as in your script with the technologies from the second script

# Creating the list for the new DataFrame
data = []
for idx, match_idx in enumerate(top_matches):
    lex_term = lex_df.iloc[idx]['Biomedical Term']
    matched_term = mrconso_df.iloc[match_idx]['STR (String)']
    similarity_score = cosine_sim[idx, match_idx]
    data.append([lex_term, matched_term, similarity_score])

# Creating the new DataFrame
new_df_columns = ['Biomedical Term', 'STR (String)', 'Cosine Similarity']
new_df = pd.DataFrame(data, columns=new_df_columns)

# Adding the 'Public Explanation' column from lex_df based on the original index
new_df['Public Explanation'] = lex_df.loc[new_df.index, 'Public Explanation']

print(new_df.head())


  Biomedical Term                                       STR (String)  \
0       Adenalgie                  Douleur d'un ganglion lymphatique   
1         Adenase                              Enzymes pancréatiques   
2     Adenectomie                  Exérèse d'un ganglion lymphatique   
3         Adenite               Inflammation de ganglion lymphatique   
4     Adenmégalie  Hypertrophie des ganglions lymph (excl lymphad...   

   Cosine Similarity           Public Explanation  
0           0.984386        Douleur aux ganglions  
1           0.977584           Enzyme du pancréas  
2           0.988938       Ablation d’un ganglion  
3           0.985442   Inflammation d’un ganglion  
4           0.983361  Élargissement d’un ganglion  


In [ ]:
data = []
for idx, cos_sim in enumerate(cosine_similaritiess):
    lex_term = lex_df.iloc[idx]['Source']
    matched_term = mrconso_df.iloc[idx]['Cible']
    data.append([lex_term, matched_term, cos_sim])

# Creating the new DataFrame with the columns in the specified order
new_df_columns = ['Source', 'Cible', 'Cosine Similarity']
new_df = pd.DataFrame(data, columns=new_df_columns)

# Formatting the 'Cosine Similarity' column to display two decimal places
new_df['Cosine Similarity'] = new_df['Cosine Similarity'].apply(lambda x: f"{x:.2f}")

print(new_df.head())

                                              Source  \
0  - Chez les patients présentant une absence com...   
1  - Eviter le contact de l'embout avec l'œil ou ...   
2  - L'utilisation prolongée de l'huile de paraff...   
3        - Ne pas dépasser la posologie recommandée.   
4  - Refermez le flacon immédiatement après utili...   

                                               Cible Cosine Similarity  
0  - Si vous savez que vous n'avez aucune activit...              0.98  
1  Evitez le contact de l'embout du flacon avec l...              0.70  
2  - L'utilisation prolongée de l'huile de paraff...              0.96  
3          Ne dépassez pas la posologie recommandée.              1.00  
4  Refermez soigneusement le flacon après utilisa...              0.99  


In [ ]:
from google.colab import files

# Saving the concatenated DataFrame to a CSV file
output_file = 'CamemSENZABATCH.csv'
new_df.to_csv(output_file, index=False)

# Download the file if you are using Google Colab
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Assuming that top_matches, cosine_sim, lex_df, and mrconso_df are already defined as in your script using the technologies from the second script

# Creating the list for the new DataFrame
data = []
for idx, match_idx in enumerate(top_matches):
    lex_term = lex_df.iloc[idx]['Biomedical Term']
    lex_explication = lex_df.iloc[idx]['Public Explanation']
    matched_term = mrconso_df.iloc[match_idx]['STR (String)']
    cui_id = mrconso_df.iloc[match_idx]['CUI (Concept Unique Identifier)']  # Aggiunta dell'Identifiant (CUI)
    similarity_score = cosine_sim[idx, match_idx]
    data.append([lex_term, lex_explication, matched_term, cui_id, similarity_score])

# Creating the new DataFrame with the columns in the specified order
new_df_columns = ['Biomedical Term', 'Public Explanation', 'STR (String)', 'CUI (Concept Unique Identifier)', 'Cosine Similarity']
new_df = pd.DataFrame(data, columns=new_df_columns)

# Formatting the 'Cosine Similarity' column to display two decimal places
new_df['Cosine Similarity'] = new_df['Cosine Similarity'].apply(lambda x: f"{x:.2f}")

print(new_df.head())

  Biomedical Term           Public Explanation  \
0       Adenalgie        Douleur aux ganglions   
1         Adenase           Enzyme du pancréas   
2     Adenectomie       Ablation d’un ganglion   
3         Adenite   Inflammation d’un ganglion   
4     Adenmégalie  Élargissement d’un ganglion   

                                        STR (String)  \
0                  Douleur d'un ganglion lymphatique   
1                              Enzymes pancréatiques   
2                  Exérèse d'un ganglion lymphatique   
3               Inflammation de ganglion lymphatique   
4  Hypertrophie des ganglions lymph (excl lymphad...   

  CUI (Concept Unique Identifier) Cosine Similarity  
0                        C0578050              0.98  
1                        C0428348              0.98  
2                        C0024203              0.99  
3                        C0024205              0.99  
4                        C0856249              0.98  


In [ ]:
import pandas as pd

# Assuming that top_matches, cosine_sim, lex_df, and mrconso_df are already defined as in your script with the technologies from the second script

# Creating the list for the new DataFrame
data = []
for idx, match_idx in enumerate(top_matches):
    # Colonne da lex_df
    lex_term = lex_df.iloc[idx]['Biomedical Term']
    lex_explication = lex_df.iloc[idx]['Public Explanation']
    lex_sources = lex_df.iloc[idx]['Sources']

    # Columns from lex_df
    columns_mrconso = [
        'LAT (Language)', 'TS (Term Status)', 'LUI (Lexical Unique Identifier)', 'STT (String Type)',
        'SUI (String Unique Identifier)', 'ISPREF (Is Preferred)', 'AUI (Atom Unique Identifier)',
        'MetaUI (Metadata Unique Identifier)', 'DescriptorUI (Descriptor Unique Identifier)',
        'SAB (Source Abbreviation)', 'TTY (Term Type)', 'Associated DescriptorUI (Descriptor Unique Identifier)',
        'STR (String)', 'Frequency', 'SUPPRESS (Suppressible Flag)'
    ]
    mrconso_values = [mrconso_df.iloc[match_idx][col] for col in columns_mrconso]

    # CUI Identifier and Cosine Similarity
    cui_id = mrconso_df.iloc[match_idx]['CUI (Concept Unique Identifier)']
    similarity_score = cosine_sim[idx, match_idx]

    # Composing the row
    row = [lex_term, lex_explication, lex_sources] + mrconso_values + [cui_id, similarity_score]
    data.append(row)

# Creating the new DataFrame with the columns in the specified order
new_df_columns = [
    'Biomedical Term', 'Public Explanation', 'Sources'
] + columns_mrconso + ['CUI (Concept Unique Identifier)', 'Cosine Similarity']

new_df = pd.DataFrame(data, columns=new_df_columns)

# Formatting the 'Cosine Similarity' column to display two decimal places
new_df['Cosine Similarity'] = new_df['Cosine Similarity'].apply(lambda x: f"{x:.2f}")

print(new_df.head())


  Biomedical Term           Public Explanation     Sources LAT (Language)  \
0       Adenalgie        Douleur aux ganglions  Affixation            FRE   
1         Adenase           Enzyme du pancréas  Affixation            FRE   
2     Adenectomie       Ablation d’un ganglion  Affixation            FRE   
3         Adenite   Inflammation d’un ganglion  Affixation            FRE   
4     Adenmégalie  Élargissement d’un ganglion  Affixation            FRE   

  TS (Term Status) LUI (Lexical Unique Identifier) STT (String Type)  \
0                P                        L5017492                PF   
1                P                        L5018432                PF   
2                S                        L5018978                PF   
3                S                       L17370370                PF   
4                P                        L6162575                PF   

  SUI (String Unique Identifier) ISPREF (Is Preferred)  \
0                       S5704997              

In [ ]:
new_df.head()

,Biomedical Term,Public Explanation,Sources,LAT (Language),TS (Term Status),LUI (Lexical Unique Identifier),STT (String Type),SUI (String Unique Identifier),ISPREF (Is Preferred),AUI (Atom Unique Identifier),MetaUI (Metadata Unique Identifier),DescriptorUI (Descriptor Unique Identifier),SAB (Source Abbreviation),TTY (Term Type),Associated DescriptorUI (Descriptor Unique Identifier),STR (String),Frequency,SUPPRESS (Suppressible Flag),CUI (Concept Unique Identifier),Similarità
0,Adenalgie,Douleur aux ganglions,Affixation,FRE,S,L6213807,PF,S7132422,Y,A27488003,M0000241,D000159,MSHFRE,ET,D000159,Huitième nerf crânien,3,N,C0001162,0.97
1,Adenase,Enzyme du pancréas,Affixation,FRE,P,L5341074,PF,S6103079,Y,A13452470,M0463736,D049148,MSHFRE,PEP,D049148,Activation des acides aminés,3,N,C0002511,0.95
2,Adenectomie,Ablation d’un ganglion,Affixation,FRE,P,L3247530,PF,S3774961,N,A24199093,NaN,10061639,MDRFRE,PT,10061639,Angiokératome,3,N,C0002985,0.97
3,Adenite,Inflammation d’un ganglion,Affixation,FRE,S,L10945458,PF,S13630910,Y,A27657958,M0000717,D000474,MSHFRE,ET,D000474,Maladie de l'urine noire,3,N,C0002066,0.96
4,Adenmégalie,Élargissement d’un ganglion,Affixation,FRE,P,L3247530,PF,S3774961,N,A24199093,NaN,10061639,MDRFRE,PT,10061639,Angiokératome,3,N,C0002985,0.97


In [ ]:
from google.colab import files

# Saving the concatenated DataFrame to a CSV file
output_file = 'Inis.csv'
new_df.to_csv(output_file, index=False)

# Download the file if you are using Google Colab
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Ensure you have imported pandas and have a DataFrame 'new_df'

# Count the rows in the DataFrame
num_righe = len(new_df)

print("Il numero totale di righe nel DataFrame 'new_df' è:", num_righe)

Il numero totale di righe nel DataFrame 'new_df' è: 16624


In [ ]:
import pandas as pd

# Loading new_df, if not already done
df_assemblage = pd.read_csv('assemblage.csv')

In [ ]:
import pandas as pd

# Loading new_df, if not already done
df_assemblage = pd.read_csv('assemblage.csv')

# Merging df_assemblage with new_df on the 'CUI' column
df_final = pd.merge(new_df, df_assemblage, on='CUI (Concept Unique Identifier)', how='left')

# Displaying the first rows of the resulting DataFrame to verify
print(df_final.head())

  Biomedical Term           Public Explanation     Sources LAT (Language)  \
0       Adenalgie        Douleur aux ganglions  Affixation            FRE   
1         Adenase           Enzyme du pancréas  Affixation            FRE   
2     Adenectomie       Ablation d’un ganglion  Affixation            FRE   
3         Adenite   Inflammation d’un ganglion  Affixation            FRE   
4     Adenmégalie  Élargissement d’un ganglion  Affixation            FRE   

  TS (Term Status) LUI (Lexical Unique Identifier) STT (String Type)  \
0                P                        L5017492                PF   
1                P                        L5018432                PF   
2                S                        L5018978                PF   
3                S                       L17370370                PF   
4                P                        L6162575                PF   

  SUI (String Unique Identifier) ISPREF (Is Preferred)  \
0                       S5704997              

In [ ]:
import pandas as pd

# Ensure you have imported pandas and have a DataFrame 'df_final'

# Count the rows in the DataFrame
num_righe = len(df_final)

print("Il numero totale di righe nel DataFrame 'df_final' è:", num_righe)

Il numero totale di righe nel DataFrame 'df_final' è: 17394


In [ ]:
df_final.head()

,Biomedical Term,Public Explanation,Sources,LAT (Language),TS (Term Status),LUI (Lexical Unique Identifier),STT (String Type),SUI (String Unique Identifier),ISPREF (Is Preferred),AUI (Atom Unique Identifier),...,CUI (Concept Unique Identifier),Cosine Similarity,TUI (Type Unique Identifier),STN (Semantic Type Tree Number),STY (Semantic Type)_x,ATUI (Attribute Unique Identifier),CVF (Content View Flag),ABBR (Abbreviation),SG (Semantic Group),STY (Semantic Type)_y
0,Adenalgie,Douleur aux ganglions,Affixation,FRE,P,L16092443,PF,S19400701,Y,A31318459,...,C0398363,0.98,T047,B2.2.1.2.1,Disease or Syndrome,AT08532420,256.0,DISO,Disorders,Disease or Syndrome
1,Adenase,Enzyme du pancréas,Affixation,FRE,S,L6148072,PF,S7070243,N,A26123888,...,C0521613,0.97,T046,B2.2.1.2,Pathologic Function,AT52812542,256.0,DISO,Disorders,Pathologic Function
2,Adenectomie,Ablation d’un ganglion,Affixation,FRE,S,L5018978,PF,S5706482,N,A26170576,...,C0024203,0.98,T061,B1.3.1.3,Therapeutic or Preventive Procedure,AT17627344,256.0,PROC,Procedures,Therapeutic or Preventive Procedure
3,Adenite,Inflammation d’un ganglion,Affixation,FRE,P,L5709844,PF,S6542881,N,A25876328,...,C0472377,0.97,T046,B2.2.1.2,Pathologic Function,AT17748708,256.0,DISO,Disorders,Pathologic Function
4,Adenmégalie,Élargissement d’un ganglion,Affixation,FRE,P,L5015528,PF,S5703033,N,A25825290,...,C0581385,0.98,T046,B2.2.1.2,Pathologic Function,AT08845483,256.0,DISO,Disorders,Pathologic Function


In [ ]:
import pandas as pd

# Assuming 'df_final' is already defined. If not, you will need to define or load it first.

# Get the list of columns
colonne = df_final.columns.tolist()

# Printing the columns
print("Elenco delle colonne in df_final:")
for colonna in colonne:
    print(colonna)

Elenco delle colonne in df_final:
Biomedical Term
Public Explanation
Sources
LAT (Language)
TS (Term Status)
LUI (Lexical Unique Identifier)
STT (String Type)
SUI (String Unique Identifier)
ISPREF (Is Preferred)
AUI (Atom Unique Identifier)
MetaUI (Metadata Unique Identifier)
DescriptorUI (Descriptor Unique Identifier)
SAB (Source Abbreviation)
TTY (Term Type)
Associated DescriptorUI (Descriptor Unique Identifier)
STR (String)
Frequency
SUPPRESS (Suppressible Flag)
CUI (Concept Unique Identifier)
Cosine Similarity
TUI (Type Unique Identifier)
STN (Semantic Type Tree Number)
STY (Semantic Type)_x
ATUI (Attribute Unique Identifier)
CVF (Content View Flag)
ABBR (Abbreviation)
SG (Semantic Group)
STY (Semantic Type)_y


In [ ]:
from google.colab import files

# Saving the concatenated DataFrame to a CSV file
output_file = 'Final_Alignment.csv'
df_final.to_csv(output_file, index=False)

# Download the file if you are using Google Colab
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>